## 1. Data 불러오기

In [1]:
import pandas as pd

In [2]:
# 데이터 불러오기
df = pd.read_csv('../../data/english_data_including_content_and_area_of_thumbnailtext.csv', usecols = [6, 8, 9], encoding='UTF-8')
print(len(df))
df.head(3)

4800


,view,thumbnail_text_content,thumbnail_text_area
0,101580,Sweets How,0.11259
1,1147867,NaN,0.00000
2,429219,NaN,0.00000


In [3]:
data_cat = df.copy()           
#print(data_cat.info())           
data_cat['views'] = (data_cat['view'].floordiv(1000))
data_cat['content'] = data_cat['thumbnail_text_content'].fillna('  ')
data_cat['area'] = (data_cat['thumbnail_text_area']*10000).astype('int64')
data_cat.drop(['view', 'thumbnail_text_content', 'thumbnail_text_area'], axis='columns', inplace=True)
data_cat.head(4)

,views,content,area
0,101,Sweets How,1125
1,1147,,0
2,429,,0
3,607,I 6AM school morning routine,1195


## 2. views preprocess
before one-hot encoding

In [4]:
data_cat['views'].max()

314441

In [5]:
import math # 구간 나눠주기 -> multiclass classification

for index, view in enumerate(data_cat['views']):
    if view >=10000:  data_cat['views'][index] =  5
    elif view >=1000:  data_cat['views'][index] =  4
    elif view >=100:  data_cat['views'][index] = 3
    elif view >=10: data_cat['views'][index] = 2
    elif view > 0: data_cat['views'][index] = 1
    else:data_cat['views'][index] = 0

<ipython-input-5-8475f35f6daf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif view >=100:  data_cat['views'][index] = 3
<ipython-input-5-8475f35f6daf>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif view >=1000:  data_cat['views'][index] =  4
<ipython-input-5-8475f35f6daf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif view >=10: data_cat['views'][index] = 2
<ipython-input-5-8475f35f6daf>:8: SettingWithCopyWarning: 

In [6]:
from collections import Counter

label = Counter(data_cat['views']).keys()
label = sorted(label)
print(label)
print(Counter(data_cat['views']))

[0, 1, 2, 3, 4, 5]
Counter({4: 1562, 3: 1504, 2: 742, 0: 522, 1: 265, 5: 205})


## 3. content preprocess (tokenization)

In [7]:
import nltk
#nltk.download('punkt')
#nltk.download('wordnet')

In [8]:
from nltk.tokenize import word_tokenize

In [9]:
data_cat['content_tokens']=None
data_cat[:3]

,views,content,area,content_tokens
0,3,Sweets How,1125,None
1,4,,0,None
2,3,,0,None


In [10]:
for index in range(len(df)):
    data_cat['content_tokens'][index] = word_tokenize(data_cat['content'][index])
data_cat[:3]

<ipython-input-10-89f23f830b8c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cat['content_tokens'][index] = word_tokenize(data_cat['content'][index])


,views,content,area,content_tokens
0,3,Sweets How,1125,"[Sweets, How]"
1,4,,0,[]
2,3,,0,[]


In [11]:
#방법1
'''
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()
data_cat['content_tokens1']=None
for index in range(len(df)):
    data_cat['content_tokens1'][index] = [lm.lemmatize(w, pos="v") for w in data_cat['content_tokens'][index]]
'''

'\nfrom nltk.stem import WordNetLemmatizer\nlm = WordNetLemmatizer()\ndata_cat[\'content_tokens1\']=None\nfor index in range(len(df)):\n    data_cat[\'content_tokens1\'][index] = [lm.lemmatize(w, pos="v") for w in data_cat[\'content_tokens\'][index]]\n'

In [12]:
#방법2
from nltk.stem import PorterStemmer
st1 = PorterStemmer()

#data_cat['content_tokens']=None

for index in range(len(df)):
    data_cat['content_tokens'][index] = [st1.stem(w) for w in data_cat['content_tokens'][index]]

<ipython-input-12-a3a3f5d3aee5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cat['content_tokens'][index] = [st1.stem(w) for w in data_cat['content_tokens'][index]]


In [13]:
#방법3
'''
from nltk.stem import LancasterStemmer
st2 = LancasterStemmer()

data_cat['content_tokens3']=None

for index in range(len(df)):
    data_cat['content_tokens3'][index] = [st2.stem(w) for w in data_cat['content_tokens'][index]]
'''

"\nfrom nltk.stem import LancasterStemmer\nst2 = LancasterStemmer()\n\ndata_cat['content_tokens3']=None\n\nfor index in range(len(df)):\n    data_cat['content_tokens3'][index] = [st2.stem(w) for w in data_cat['content_tokens'][index]]\n"

In [14]:
#방법 1,2,3 중 선택
'''
data_cat['content_tokens']=data_cat['content_tokens2']
data_cat.drop(['content', 'content_tokens1', 'content_tokens2', 'content_tokens3'], axis='columns', inplace=True)
data_cat[:3]
'''

"\ndata_cat['content_tokens']=data_cat['content_tokens2']\ndata_cat.drop(['content', 'content_tokens1', 'content_tokens2', 'content_tokens3'], axis='columns', inplace=True)\ndata_cat[:3]\n"

In [15]:
data_cat[:4]

,views,content,area,content_tokens
0,3,Sweets How,1125,"[sweet, how]"
1,4,,0,[]
2,3,,0,[]
3,3,I 6AM school morning routine,1195,"[i, 6am, school, morn, routin]"


## 4. area preprocess

In [16]:
data_cat['area'].max()

11941

In [17]:
import math # 구간 나눠주기 -> multiclass classification

for index, area in enumerate(data_cat['area']):
    if area >=3000:  data_cat['area'][index] =  4
    elif area >=1000:  data_cat['area'][index] =3
    elif area >=100:  data_cat['area'][index] = 2
    elif area >=1: data_cat['area'][index] = 1
    else : data_cat['area'][index] = 0

<ipython-input-17-148d99b600fd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif area >=1000:  data_cat['area'][index] =3
<ipython-input-17-148d99b600fd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else : data_cat['area'][index] = 0
<ipython-input-17-148d99b600fd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif area >=100:  data_cat['area'][index] = 2
<ipython-input-17-148d99b600fd>:7: SettingWithCopyWarning: 
A value i

In [18]:
from collections import Counter

label = Counter(data_cat['area']).keys()
label = sorted(label)
print(label)
print(Counter(data_cat['area']))

[0, 1, 2, 3, 4]
Counter({3: 1869, 2: 1417, 0: 1110, 4: 260, 1: 144})


## 5. make csv file

In [19]:
data_cat.to_csv("../../data/english_thumbnailtext_preprocessed.csv", header=True, index=False)
data_cat

,views,content,area,content_tokens
0,3,Sweets How,3,"[sweet, how]"
1,4,,0,[]
2,3,,0,[]
3,3,I 6AM school morning routine,3,"[i, 6am, school, morn, routin]"
4,2,,0,[]
...,...,...,...,...
4795,4,,0,[]
4796,3,,0,[]
4797,2,a - OPPO Review,3,"[a, -, oppo, review]"
4798,1,15 RANDOM FACTS about ME,3,"[15, random, fact, about, me]"
